## This is a demonstration of the generation of virtualzarr
Reference   https://virtualizarr.readthedocs.io/en/stable/index.html#features

1. specific examples to show how metadata changes can be made lightweight in the side car file alone and written to an icechunk store

In [27]:
import xarray as xr
from obstore.store import from_url
from virtualizarr import open_virtual_dataset, open_virtual_mfdataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry
import icechunk, kerchunk, parquet

In [28]:
#%pip install obstore

In [29]:
#%pip install virtualizarr icechunk

In [30]:
#%pip install --upgrade xarray

In [31]:
#%pip install icechunk

In [32]:
#%pip install kerchunk

In [33]:
#%pip install fastparquet

In [34]:
#Zarr can emit a lot of warnings about Numcodecs not being including in 
#the Zarr version 3 specification yet -- let's suppress those.

import warnings
warnings.filterwarnings(
  "ignore",
  message="Numcodecs codecs are not in the Zarr version 3 specification*",
  category=UserWarning
)

In [62]:
#We can use Obstore's obstore.store.from_url convenience method to create an 
#ObjectStore that can fetch data from the specified URLs.
#https://noaa-gfdl-spear-large-ensembles-pds.s3.amazonaws.com/index.html
bucket = "s3://noaa-gfdl-spear-large-ensembles-pds"
path = "SPEAR/GFDL-LARGE-ENSEMBLES/CMIP/NOAA-GFDL/GFDL-SPEAR-MED/historical/r10i1p1f1/Amon/tas/gr3/v20210201/tas_Amon_GFDL-SPEAR-MED_historical_r10i1p1f1_gr3_192101-201412.nc"
url = f"{bucket}/{path}"
store = from_url(bucket, region="us-east-1", skip_signature=True)

#We also need to create an ObjectStoreRegistry 
#that maps the URL structure to the ObjectStore.

registry = ObjectStoreRegistry({bucket: store})


In [63]:
url

's3://noaa-gfdl-spear-large-ensembles-pds/SPEAR/GFDL-LARGE-ENSEMBLES/CMIP/NOAA-GFDL/GFDL-SPEAR-MED/historical/r10i1p1f1/Amon/tas/gr3/v20210201/tas_Amon_GFDL-SPEAR-MED_historical_r10i1p1f1_gr3_192101-201412.nc'

In [36]:
#Now, let's create a parser instance and create a virtual dataset by passing the URL, 
#parser, and registry to virtualizarr.open_virtual_dataset.

In [37]:
parser = HDFParser()
vds = open_virtual_dataset(
    url=url,
    registry=registry,
    parser=parser,
    loadable_variables=['time'],
    decode_times=True
)
print(vds)

<xarray.Dataset> Size: 936MB
Dimensions:    (time: 1128, lat: 360, bnds: 2, lon: 576)
Coordinates:
  * time       (time) object 9kB 1921-01-16 12:00:00 ... 2014-12-16 12:00:00
    bnds       (bnds) float64 16B ManifestArray<shape=(2,), dtype=float64, ch...
    lat        (lat) float64 3kB ManifestArray<shape=(360,), dtype=float64, c...
    lon        (lon) float64 5kB ManifestArray<shape=(576,), dtype=float64, c...
Data variables:
    lat_bnds   (lat, bnds) float64 6kB ManifestArray<shape=(360, 2), dtype=fl...
    lon_bnds   (lon, bnds) float64 9kB ManifestArray<shape=(576, 2), dtype=fl...
    tas        (time, lat, lon) float32 936MB ManifestArray<shape=(1128, 360,...
    time_bnds  (time, bnds) float64 18kB ManifestArray<shape=(1128, 2), dtype...
Attributes: (12/28)
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               Amon
    contact:                gfdl.climate.model.info@noaa.gov
    experiment:             all-forcing simula

In [38]:
type(vds)

xarray.core.dataset.Dataset

In [39]:
vds.tas

<xarray.DataArray 'tas' (time: 1128, lat: 360, lon: 576)> Size: 936MB
ManifestArray<shape=(1128, 360, 576), dtype=float32, chunks=(1, 360, 576)>
Coordinates:
  * time     (time) object 9kB 1921-01-16 12:00:00 ... 2014-12-16 12:00:00
    lat      (lat) float64 3kB ManifestArray<shape=(360,), dtype=float64, chu...
    lon      (lon) float64 5kB ManifestArray<shape=(576,), dtype=float64, chu...
Attributes:
    long_name:      Near-Surface Air Temperature
    units:          K
    valid_range:    [100.0, 400.0]
    missing_value:  1.0000000200408773e+20
    _FillValue:     AAAAgB2vFUQ=
    cell_methods:   time: mean
    interp_method:  conserve_order2
    standard_name:  air_temperature
    cell_measures:  area: areacella

In [40]:
vds.attrs["comments"] = "vds creation for prototyping"

In [41]:
vds.tas.attrs["long_name"] = "Near-Surface air temperature"

In [42]:
vds

<xarray.Dataset> Size: 936MB
Dimensions:    (time: 1128, lat: 360, bnds: 2, lon: 576)
Coordinates:
  * time       (time) object 9kB 1921-01-16 12:00:00 ... 2014-12-16 12:00:00
    bnds       (bnds) float64 16B ManifestArray<shape=(2,), dtype=float64, ch...
    lat        (lat) float64 3kB ManifestArray<shape=(360,), dtype=float64, c...
    lon        (lon) float64 5kB ManifestArray<shape=(576,), dtype=float64, c...
Data variables:
    lat_bnds   (lat, bnds) float64 6kB ManifestArray<shape=(360, 2), dtype=fl...
    lon_bnds   (lon, bnds) float64 9kB ManifestArray<shape=(576, 2), dtype=fl...
    tas        (time, lat, lon) float32 936MB ManifestArray<shape=(1128, 360,...
    time_bnds  (time, bnds) float64 18kB ManifestArray<shape=(1128, 2), dtype...
Attributes: (12/29)
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               Amon
    contact:                gfdl.climate.model.info@noaa.gov
    experiment:             all-forcing simulation for 1921 to 2014
    experiment_id:          historical
    Conventions:            CF-1.7
    ...                     ...
    title:                  NOAA GFDL GFDL-SPEAR-MED, all-forcing simulation ...
    gfdl_experiment_name:   SPEAR-MED.historical.r10
    license:                CC-BY-SA-4.0
    forcing:                CMIP6 historical
    external_variables:     areacella
    comments:               vds creation for prototyping

In [43]:
vds.tas

<xarray.DataArray 'tas' (time: 1128, lat: 360, lon: 576)> Size: 936MB
ManifestArray<shape=(1128, 360, 576), dtype=float32, chunks=(1, 360, 576)>
Coordinates:
  * time     (time) object 9kB 1921-01-16 12:00:00 ... 2014-12-16 12:00:00
    lat      (lat) float64 3kB ManifestArray<shape=(360,), dtype=float64, chu...
    lon      (lon) float64 5kB ManifestArray<shape=(576,), dtype=float64, chu...
Attributes:
    long_name:      Near-Surface air temperature
    units:          K
    valid_range:    [100.0, 400.0]
    missing_value:  1.0000000200408773e+20
    _FillValue:     AAAAgB2vFUQ=
    cell_methods:   time: mean
    interp_method:  conserve_order2
    standard_name:  air_temperature
    cell_measures:  area: areacella

In [44]:
#Since we specified loadable_variables=[], 
#no data has been loaded or copied in this process. 
#We have merely created an in-memory lookup table that points to the
#location of chunks in the original netCDF when data is needed later on. 
#The default behavior (loadable_variables=None) will load data associated with 
#coordinates but not data variables. The size represents the size of the original 
#dataset - you can see the size of the virtual dataset using the vz accessor:

In [45]:
print(f"Original dataset size: {vds.nbytes} bytes")
print(f"Virtual dataset size: {vds.vz.nbytes} bytes")

Original dataset size: 935657872 bytes
Virtual dataset size: 81376 bytes


In [46]:
#VirtualiZarr's other top-level function is virtualizarr.open_virtual_mfdataset, which can open and virtualize multiple data sources into a single virtual dataset, similar to how xarray.open_mfdataset 
#opens multiple data files as a single dataset.

Combining virtual datasets¶
In general we should be able to combine all the datasets from our archival files into one using some combination of calls to xarray.concat and xarray.merge. For combining along multiple dimensions in one call we also have xarray.combine_nested and xarray.combine_by_coords. If you're not familiar with any of these functions we recommend you skim through xarray's docs on combining.

You can achieve both the opening and combining steps for multiple files in one go 
by using open_virtual_mfdataset.


Ordering by coordinate values¶
If you're happy to load 1D dimension coordinates into memory, you can use their values to do the ordering for you!

In [47]:
#The magic of VirtualiZarr is that you can 
#persist the virtual dataset to disk in a chunk references format such as Icechunk, 
#meaning that the work of constructing the single coherent dataset only needs to happen 
#once. For subsequent data access, you can use xarray.open_zarr to open that Icechunk 
#store, which on object storage is far faster than using xarray.open_mfdataset to 
#open the the original non-cloud-optimized files.

In [48]:
#Let's persist the Virtual dataset using Icechunk. 
#First let's create an Icechunk configuration with permissions to access our data.

In [49]:
#Now we can store the references to our data. 
#Here we store the references in an icechunk store 
#that only lives in memory, 
#but in most cases you'll store the "virtual" icechunk store in the cloud.

In [52]:
storage = icechunk.local_filesystem_storage(
    path='spear-tas-metadata2',
)
config = icechunk.RepositoryConfig.default()
config.set_virtual_chunk_container(icechunk.VirtualChunkContainer("s3://noaa-gfdl-spear-large-ensembles-pds/", icechunk.s3_store(region="us-east-1")))
credentials = icechunk.containers_credentials({"s3://noaa-gfdl-spear-large-ensembles-pds/": icechunk.s3_credentials(anonymous=True)})
repo = icechunk.Repository.create(storage, config, credentials)

  2025-10-03T18:40:05.819807Z  WARN icechunk::storage::object_store: The LocalFileSystem storage is not safe for concurrent commits. If more than one thread/process will attempt to commit at the same time, prefer using object stores.
    at icechunk/src/storage/object_store.rs:80



In [53]:
session = repo.writable_session("main")
vds.virtualize.to_icechunk(session.store)

In [54]:
session.commit("Metadata edited in vds virtual store!")

'H46Y527M1JHN69XF2RR0'

In [55]:
ds = xr.open_zarr(
    session.store,
    zarr_version=3,
    consolidated=False,
    chunks={},
)

/vftmp/Aparna.Radhakrishnan/pid731721/ipykernel_1120947/2050398136.py:1: FutureWarning: zarr_version is deprecated, use zarr_format
  ds = xr.open_zarr(


In [ ]:
ds

In [ ]:
ds.tas.isel(time=0) #yay!

In [ ]:
ds.tas.isel(time=0).plot()

In [ ]:
#We can also use it with our analysis scripts by writing to kerchunk's format instead of icechunk

In [ ]:
ts = ds.tas.sel(lat=0, lon=180, method="nearest")
ts.plot()

# let's create a store with ALL ensemble members

In [ ]:
url_rip_Amon_tas = "s3://noaa-gfdl-spear-large-ensembles-pds/SPEAR/GFDL-LARGE-ENSEMBLES/CMIP/NOAA-GFDL/GFDL-SPEAR-MED/historical/*/Amon/tas/gr3/v20210201/*.nc"


In [2]:
import s3fs
import re

def get_run_sorted_files(variable="tas", bucket="noaa-gfdl-spear-large-ensembles-pds"):
    """
    Return a list of S3 URLs for a given variable (tas, pr, etc.)
    from the NOAA GFDL-SPEAR-MED historical ensemble, sorted by r* member number.
    """
    fs = s3fs.S3FileSystem(anon=True)

    prefix = "SPEAR/GFDL-LARGE-ENSEMBLES/CMIP/NOAA-GFDL/GFDL-SPEAR-MED/historical/"
    all_keys = fs.find(f"{bucket}/{prefix}")

    
    # Filter only matching variable NetCDFs
  #  files = [f"s3://{k}" for k in all_keys if f"/Amon/{variable}/" in k and k.endswith(".nc")]
    #files = [f"s3://{bucket}/{k}" for k in all_keys if f"/Amon/{variable}/" in k and k.endswith(".nc")]
    files = [f"s3://{k}" for k in all_keys if f"/Amon/{variable}/" in k and k.endswith(".nc")]

    # Sort by ensemble member r*
    def run_sort_key(path):
        fname = path.split("/")[-1]
        match = re.search(r"r(\d+)i", fname)
        return int(match.group(1)) if match else 9999

    return sorted(files, key=run_sort_key)


In [3]:
tas_files = get_run_sorted_files("tas")
#print(len(tas_files), "tas files")
print(tas_files[:1])

['s3://noaa-gfdl-spear-large-ensembles-pds/SPEAR/GFDL-LARGE-ENSEMBLES/CMIP/NOAA-GFDL/GFDL-SPEAR-MED/historical/r1i1p1f1/Amon/tas/gr3/v20210201/tas_Amon_GFDL-SPEAR-MED_historical_r1i1p1f1_gr3_192101-201412.nc']


In [ ]:
#%pip install "s3fs==2024.6.1" "aiobotocore==2.15.2" "botocore==1.35.16" "xarray"

In [88]:
store = from_url(bucket, region="us-east-1", skip_signature=True)

#We also need to create an ObjectStoreRegistry 
#that maps the URL structure to the ObjectStore.

registry = ObjectStoreRegistry({bucket: store})
parser = HDFParser()
#url='s3://noaa-gfdl-spear-large-ensembles-pds/SPEAR/GFDL-LARGE-ENSEMBLES/CMIP/NOAA-GFDL/GFDL-SPEAR-MED/historical/r1i1p1f1/Amon/tas/gr3/v20210201/tas_Amon_GFDL-SPEAR-MED_historical_r1i1p1f1_gr3_192101-201412.nc'
#noaa-gfdl-spear-large-ensembles-pds/SPEAR/GFDL-LARGE-ENSEMBLES/CMIP/NOAA-GFDL/GFDL-SPEAR-MED/historical/r10i1p1f1/Amon/tas/gr3/v20210201/tas_Amon_GFDL-SPEAR-MED_historical_r10i1p1f1_gr3_192101-201412.nc'
virtual_datasets = [
  open_virtual_dataset(
    url2,
    registry=registry,
    parser=parser,
    loadable_variables=[],
    decode_times=True)
  for url2 in tas_files
]

In [94]:
virtual_datasets[10]

<xarray.Dataset> Size: 936MB
Dimensions:    (lat: 360, bnds: 2, lon: 576, time: 1128)
Coordinates:
    bnds       (bnds) float64 16B ManifestArray<shape=(2,), dtype=float64, ch...
    lat        (lat) float64 3kB ManifestArray<shape=(360,), dtype=float64, c...
    lon        (lon) float64 5kB ManifestArray<shape=(576,), dtype=float64, c...
    time       (time) float64 9kB ManifestArray<shape=(1128,), dtype=float64,...
Data variables:
    lat_bnds   (lat, bnds) float64 6kB ManifestArray<shape=(360, 2), dtype=fl...
    lon_bnds   (lon, bnds) float64 9kB ManifestArray<shape=(576, 2), dtype=fl...
    tas        (time, lat, lon) float32 936MB ManifestArray<shape=(1128, 360,...
    time_bnds  (time, bnds) float64 18kB ManifestArray<shape=(1128, 2), dtype...
Attributes: (12/28)
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:               Amon
    contact:                gfdl.climate.model.info@noaa.gov
    experiment:             all-forcing simulation for 1921 to 2014
    experiment_id:          historical
    Conventions:            CF-1.7
    ...                     ...
    references:             SPEAR: The next generation GFDL modeling system f...
    title:                  NOAA GFDL GFDL-SPEAR-MED, all-forcing simulation ...
    gfdl_experiment_name:   SPEAR-MED.historical.r11
    license:                CC-BY-SA-4.0
    forcing:                CMIP6 historical
    external_variables:     areacella

In [4]:
def extract_member(path):
    m = re.search(r"(r\d+i\d+p\d+f\d+)", path)
    return m.group(1) if m else "unknown"

# Extract member IDs for coordinate labeling
members = [extract_member(f) for f in tas_files]
print(members)

['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1', 'r4i1p1f1', 'r5i1p1f1', 'r6i1p1f1', 'r7i1p1f1', 'r8i1p1f1', 'r9i1p1f1', 'r10i1p1f1', 'r11i1p1f1', 'r12i1p1f1', 'r13i1p1f1', 'r14i1p1f1', 'r15i1p1f1', 'r16i1p1f1', 'r17i1p1f1', 'r18i1p1f1', 'r19i1p1f1', 'r20i1p1f1', 'r21i1p1f1', 'r22i1p1f1', 'r23i1p1f1', 'r24i1p1f1', 'r25i1p1f1', 'r26i1p1f1', 'r27i1p1f1', 'r28i1p1f1', 'r29i1p1f1', 'r30i1p1f1']


In [103]:
# Open them all together
vds_tas_ens = xr.open_mfdataset(
    tas_files,
    engine="h5netcdf",            
    backend_kwargs={
        "storage_options": {"anon": True},   
    },
    combine="nested",
    concat_dim="member",
    parallel=True,
)

# Label the ensemble dimension
vds_tas_ens = vds_tas_ens.assign_coords(member=("member", members))
print(vds_tas_ens)

<xarray.Dataset> Size: 28GB
Dimensions:    (member: 30, lat: 360, bnds: 2, lon: 576, time: 1128)
Coordinates:
  * bnds       (bnds) float64 16B 1.0 2.0
  * lat        (lat) float64 3kB -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * lon        (lon) float64 5kB 0.3125 0.9375 1.562 2.188 ... 358.4 359.1 359.7
  * time       (time) object 9kB 1921-01-16 12:00:00 ... 2014-12-16 12:00:00
  * member     (member) <U9 1kB 'r1i1p1f1' 'r2i1p1f1' ... 'r30i1p1f1'
Data variables:
    lat_bnds   (member, lat, bnds) float64 173kB dask.array<chunksize=(1, 360, 2), meta=np.ndarray>
    lon_bnds   (member, lon, bnds) float64 276kB dask.array<chunksize=(1, 576, 2), meta=np.ndarray>
    tas        (member, time, lat, lon) float32 28GB dask.array<chunksize=(1, 1, 360, 576), meta=np.ndarray>
    time_bnds  (member, time, bnds) object 541kB dask.array<chunksize=(1, 1, 2), meta=np.ndarray>
Attributes: (12/28)
    history:                File was processed by fremetar (GFDL analog of CM...
    table_id:       

In [105]:
#slowwwww
storage = icechunk.local_filesystem_storage(
    path='spear-Amon-tas-ripf',
)
config = icechunk.RepositoryConfig.default()
config.set_virtual_chunk_container(icechunk.VirtualChunkContainer("s3://noaa-gfdl-spear-large-ensembles-pds/", icechunk.s3_store(region="us-east-1")))
credentials = icechunk.containers_credentials({"s3://noaa-gfdl-spear-large-ensembles-pds/": icechunk.s3_credentials(anonymous=True)})
repo = icechunk.Repository.create(storage, config, credentials)
session = repo.writable_session("main")
vds_tas_ens.virtualize.to_icechunk(session.store)

session.commit("Metadata edited in vds virtual store!")

ds = xr.open_zarr(
    session.store,
    zarr_version=3,
    consolidated=False,
    chunks={},
)

  2025-10-03T19:50:16.106137Z  WARN icechunk::storage::object_store: The LocalFileSystem storage is not safe for concurrent commits. If more than one thread/process will attempt to commit at the same time, prefer using object stores.
    at icechunk/src/storage/object_store.rs:80



/nbhome/Aparna.Radhakrishnan/conda/envs/catalogbuilder/lib/python3.12/site-packages/virtualizarr/accessor.py:39: UserWarning: Attempting to write an entirely non-virtual Dataset to a virtual references format - i.e. your `xarray.Dataset` contains zero `ManifestArray` objects. This is almost certainly not intended, as the entire data contents will be duplicated rather than referenced. This may have happened because you used `xarray.open_Dataset` instead of `virtualizarr.open_virtual_Dataset`, or you set all variables to be `loadable_variables`.Please read the usage docs.
  warnings.warn(
/nbhome/Aparna.Radhakrishnan/conda/envs/catalogbuilder/lib/python3.12/site-packages/zarr/core/dtype/npy/string.py:248: UnstableSpecificationWarning: The data type (FixedLengthUTF32(length=9, endianness='little')) does not have a Zarr V3 specification. That means that the representation of arrays saved with this data type may change without warning in a future version of Zarr Python. Arrays stored with t

# lets try a difrnt workflow

In [37]:
#see other notebooks kerchunk*
#%pip install --upgrade "fsspec>=2024.2.0" "s3fs>=2024.2.0" "kerchunk>=0.2.6"
#%pip install --upgrade "fsspec>=2024.2.0" "s3fs>=2024.2.0" "xarray>=2024.1.0" "kerchunk>=0.2.6"


In [ ]:
ds